In [1]:
import numpy as np
from scipy.integrate import solve_ivp
from time import time
import matplotlib.pyplot as plt
import pandas as pd
import math
from scipy.optimize import minimize, curve_fit

# import cupy as cp

In [2]:
t_min = 0
t_end = 24 * 10

In [3]:
data = pd.read_csv('experiment.csv')
data = [[0] + data[str(i)].to_list()[:6] for i in range(1, 4)]
data = [[data[0][i], data[1][i], data[2][i]] for i in range(6)]
data = np.cumsum(data, axis=0)

mean_srip_data = np.mean(data, axis=1)  # 各時間点での平均値
std_srip_data = np.std(data, axis=1)  # 各時間点での標準偏差

t_data = np.array([24*i for i in range(6)])

In [5]:
# def model(t, y, P, X_max, Ser, K_ser, k_DNA, k_sRNA, k_dRNA, k_vp, K_RNA, k_t):
#     X, DNA, RNA, SRIP = y

#     dXdt = P * X * (X_max - X) / X_max * Ser / (Ser + K_ser) - k_t * SRIP
#     dDNAdt = -k_DNA * DNA
#     dRNAdt = k_sRNA * DNA - k_dRNA * RNA
#     dSRIPdt = k_vp * RNA / (RNA + K_RNA) * X
    
#     if X < 0:
#         X = 0
#     return [dXdt, dDNAdt, dRNAdt, dSRIPdt]

In [6]:
# params_fix = [
#     # P,             X_max, Ser, k_DNA,        
#     [math.log(2)/24, 1.3e6, 42,   math.log(2)/24]
#     ]

# params = [
#     #K_ser, k_sRNA, k_dRNA, k_vp, K_RNA, k_t
#     [1,     1,      1,      1,    1,     1],
#     ]

# X_0 = 2e5
# DNA_0 = 2.0e11
# DNA_0 = DNA_0 / X_0 * 0.8
# RNA_0 = 0.0  # Okumura
# SRIP_0 = 0.0  # SRIPの初期値を適切に設定してください
# y0 = [X_0, DNA_0, RNA_0, SRIP_0] # X, DNA, RNA, SRIP

In [7]:
# results = []

# fig = plt.figure()
# ax = fig.add_subplot(1, 1, 1)

# for param in params:
#     P, X_max, Ser, k_DNA = params_fix[0]
#     K_ser, k_sRNA, k_dRNA, k_vp, K_RNA, k_t = param
    
#     results.append(solve_ivp(model, [t_min, t_end], y0, args=(P, X_max, Ser, K_ser, k_DNA, k_sRNA, k_dRNA, k_vp, K_RNA, k_t), t_eval=np.linspace(t_min, t_end, 1000), method='RK45'))
    
#     ax.plot(results[-1].t, results[-1].y[3], label=f'param: params={K_ser, k_sRNA, k_dRNA, k_vp, K_RNA, k_t}')


# for i in range(6):
#     datum = data[i]
#     for j in range(len(data[i])):
#         ax.plot(24 * i, datum[j], '.', color='black')

# ax.errorbar(t_data, mean_srip_data, yerr=std_srip_data, fmt='o', label='Experimental - SRIP', color='gray')

# ax.set_title('Effect of X_max on SRIP')
# ax.set_ylim(0, 5e5)
# ax.legend()

# curve_fitを使用

In [4]:
def model_for_fit(t, y, K_ser, k_sRNA, k_dRNA, k_vp, K_RNA, k_t):
    X, DNA, RNA, SRIP = y
    # X = max(X, 0)
    # DNA = max(DNA, 0)
    # RNA = max(RNA, 0)
    # SRIP = max(SRIP, 0)
    
    dXdt = P * X * (X_max - X) / X_max * Ser / (Ser + K_ser) - k_t * SRIP
    dDNAdt = -k_DNA * DNA
    dRNAdt = k_sRNA * DNA - k_dRNA * RNA
    dSRIPdt = k_vp * RNA / (RNA + K_RNA) * X
    
    # print(dXdt, max(dXdt, 0))
    # dXdt = max(dXdt, 0)
    # dDNAdt = max(dDNAdt, 0)
    # dRNAdt = max(dRNAdt, 0)
    # dSRIPdt = max(dSRIPdt, 0)
    
    return [dXdt, dDNAdt, dRNAdt, dSRIPdt]

In [6]:
params_fix = [
    # P,             X_max, Ser,    k_DNA,        
    [math.log(2)/24, 1.3e6, 4.0e-4, math.log(2)/24]
    ]

initial_fitting_param = [
    # 初期値は論文の値を参照
    #K_Ser,  k_sRNA, k_dRNA, k_vp, K_RNA, k_t
    [1.0e-5, 3e-2,   3e-2,   1e-2, 2e+3,  1e-2]
    #fix,    fix,    fix,    TBD,  fix,   TBD
    ]

bounds = [(1.0e-8, 1.0e-2), (3.0e-5, 3.0e+1), (3.0e-5, 3.0e+1), (1.0e-5, 1.0e+1), (2.0e+0, 2.0e+6), (1.0e-5, 1.0e+1)]

P, X_max, Ser, k_DNA = params_fix[0]

X_0 = 2e5
DNA_0 = 2.0e11
RNA_0 = 0.0
SRIP_0 = 0.0
initial_value = [X_0, DNA_0 / X_0 * 0.8, RNA_0, SRIP_0] # X, DNA, RNA, SRIP
print(initial_value)

[200000.0, 800000.0, 0.0, 0.0]


In [7]:
def fit_function(t, K_ser, k_sRNA, k_dRNA, k_vp, K_RNA, k_t):
    sol = solve_ivp(model_for_fit, [t_min, t_end], initial_value, args=(K_ser, k_sRNA, k_dRNA, k_vp, K_RNA, k_t), t_eval=t, method='RK45')
    
    return sol.y[-1]

popt_clip, _ = curve_fit(fit_function, t_data, (data[:, 0] + data[:, 2])/2, p0=initial_fitting_param[0], bounds=np.array(bounds).T)
popt_full, _ = curve_fit(fit_function, t_data, mean_srip_data, p0=initial_fitting_param[0], bounds=np.array(bounds).T)
popt_clip_not_bounds, _ = curve_fit(fit_function, t_data, (data[:, 0] + data[:, 2])/2, p0=initial_fitting_param[0])
popt_full_not_bounds, _ = curve_fit(fit_function, t_data, mean_srip_data, p0=initial_fitting_param[0])

# print('initial:', initial_fitting_param)
print('最適化されたパラメータ(full):', popt_full_not_bounds)

results = []

K_ser, k_sRNA, k_dRNA, k_vp, K_RNA, k_t = popt_clip
results.append(solve_ivp(model_for_fit, [t_min, t_end], initial_value, args=(K_ser, k_sRNA, k_dRNA, k_vp, K_RNA, k_t), t_eval=np.linspace(t_min, t_end, 1000), method='RK45'))

K_ser, k_sRNA, k_dRNA, k_vp, K_RNA, k_t = popt_full
results.append(solve_ivp(model_for_fit, [t_min, t_end], initial_value, args=(K_ser, k_sRNA, k_dRNA, k_vp, K_RNA, k_t), t_eval=np.linspace(t_min, t_end, 1000), method='RK45'))

K_ser, k_sRNA, k_dRNA, k_vp, K_RNA, k_t = popt_clip_not_bounds
results.append(solve_ivp(model_for_fit, [t_min, t_end], initial_value, args=(K_ser, k_sRNA, k_dRNA, k_vp, K_RNA, k_t), t_eval=np.linspace(t_min, t_end, 1000), method='RK45'))

K_ser, k_sRNA, k_dRNA, k_vp, K_RNA, k_t = popt_full_not_bounds
results.append(solve_ivp(model_for_fit, [t_min, t_end], initial_value, args=(K_ser, k_sRNA, k_dRNA, k_vp, K_RNA, k_t), t_eval=np.linspace(t_min, t_end, 1000), method='RK45'))

fig = plt.figure(figsize=(12, 8))
ax = fig.add_subplot(1, 3, 1)
ax.plot(results[0].t[results[0].y[0] >= 0], results[0].y[3][results[0].y[0] >= 0], label=f'SRIP, n=1, 3 (with bound)')
ax.plot(results[1].t[results[1].y[0] >= 0], results[1].y[3][results[1].y[0] >= 0], label=f'SRIP, n=1, 2, 3 (with bound)')
ax.plot(results[2].t[results[2].y[0] >= 0], results[2].y[3][results[2].y[0] >= 0], label=f'SRIP, n=1, 3')
ax.plot(results[3].t[results[3].y[0] >= 0], results[3].y[3][results[3].y[0] >= 0], label=f'SRIP, n=1, 2, 3')

for i in range(6):
    datum = data[i]
    for j in range(len(data[i])):
        ax.plot(24 * i, datum[j], '.', color='black')
ax.errorbar(t_data, mean_srip_data, yerr=std_srip_data, fmt='o', label='Experimental - SRIP', color='gray')
ax.set_title('Effect of Ks on SRIP')
# ax.legend()

ax = fig.add_subplot(1, 3, 2)
ax.plot(results[0].t[results[0].y[0] >= 0], results[0].y[0][results[0].y[0] >= 0], label=f'cell, n=1, 3 (with bound)')
ax.plot(results[1].t[results[1].y[0] >= 0], results[1].y[0][results[1].y[0] >= 0], label=f'cell, n=1, 2, 3 (with bound)')
ax.plot(results[2].t[results[2].y[0] >= 0], results[2].y[0][results[2].y[0] >= 0], label=f'cell, n=1, 3')
ax.plot(results[3].t[results[3].y[0] >= 0], results[3].y[0][results[3].y[0] >= 0], label=f'cell, n=1, 2, 3')
# ax.legend()
# ax.axhline(y=0)

ax = fig.add_subplot(1, 3, 3)
ax.plot(results[0].t[results[0].y[0] >= 0], results[0].y[2][results[0].y[0] >= 0], label=f'RNA, n=1, 3 (with bound)')
ax.plot(results[1].t[results[1].y[0] >= 0], results[1].y[2][results[1].y[0] >= 0], label=f'RNA, n=1, 2, 3 (with bound)')
ax.plot(results[2].t[results[2].y[0] >= 0], results[2].y[2][results[2].y[0] >= 0], label=f'RNA, n=1, 3')
ax.plot(results[3].t[results[3].y[0] >= 0], results[3].y[2][results[3].y[0] >= 0], label=f'RNA, n=1, 2, 3')
# ax.axhline(y=0)
ax.set_ylim(0, 5e5)
ax.legend()

/usr/local/lib/python3.9/site-packages/scipy/optimize/_minpack_py.py:1010: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/tmp/ipykernel_2265/4252517653.py:11: RuntimeWarning: invalid value encountered in scalar divide
  dSRIPdt = k_vp * RNA / (RNA + K_RNA) * X


ValueError: operands could not be broadcast together with shapes (5,) (6,) 

In [74]:
const_name = ['K_ser', 'k_sRNA', 'k_dRNA', 'k_vp', 'K_RNA', 'k_t']
for i in range(popt_full.shape[0]):
    # print(popt_full[i])
    print(f'{const_name[i]} = {popt_full[i]:.1e}')
print([f'{popt_full[i]:.1e}' for i in range(len(const_name))])

K_ser = 1.0e-06
k_sRNA = 1.3e-02
k_dRNA = 1.4e+00
k_vp = 1.9e-02
K_RNA = 2.9e+03
k_t = 4.3e-02
['1.0e-06', '1.3e-02', '1.4e+00', '1.9e-02', '2.9e+03', '4.3e-02']


# 初期値を探索する

In [226]:
# 1. 各パラメータの探索範囲を定義
step_size = 4
K_ser_range = np.linspace(1e-3, 1e-2, step_size)
k_sRNA_range = np.linspace(1e-1, 1e+0, step_size)
k_dRNA_range = np.linspace(1e-3, 3e-2, step_size)
k_vp_range = np.linspace(1e-2, 5e-2, step_size)
K_RNA_range = np.linspace(1e-2, 1e+1, step_size)
k_t_range = np.linspace(1e-3, 1e-1, step_size) 

cnt = 0

# 最良のフィットを追跡するための変数
best_fit = np.inf
best_params = None

# 2. 全ての組み合わせに対してモデルのフィットを実行
for K_ser in K_ser_range:
    for k_sRNA in k_sRNA_range:
        for k_dRNA in k_dRNA_range:
            for k_vp in k_vp_range:
                for K_RNA in K_RNA_range:
                    for k_t in k_t_range:
                        p0_grid = [K_ser, k_sRNA, k_dRNA, k_vp, K_RNA, k_t]  # 現在の組み合わせを初期値として使用
                        popt, pcov = curve_fit(fit_function, t_data, data[:, 0], p0=p0_grid, bounds=np.array(bounds).T)

                        # 現在のフィットの良さを計算（例: 残差平方和）
                        residual = np.sum((data[:, 0] - fit_function(t_data, *popt))**2)

                        # 最良のフィットを更新
                        if residual < best_fit:
                            best_fit = residual
                            best_params = popt
                        
                        cnt += 1
                        print(cnt) if cnt % 10 == 0 else None

# 3. 最も良いフィットを示すパラメータの組み合わせを出力
print("Best fit parameters:", best_params)

10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190


RuntimeError: Optimal parameters not found: The maximum number of function evaluations is exceeded.

# differential_evolution

In [130]:
# def objective_function(params):
#     sol = solve_ivp(model_for_fit, [t_min, t_end], y0, args=tuple(params), t_eval=t_data, method='RK45')
#     # 二乗誤差を計算
#     return np.sum((sol.y[-1] - data[:, 0])**2)

# bounds = [(0, 1e2), (0, 1e2), (0, 1e2), (0, 1e2), (0, 1e2), (0, 1e2)]
# result = differential_evolution(objective_function, bounds)


# optimized_params = result.x
# print("最適化されたパラメータ:", optimized_params)